## ECE-505 Computer Project 1
### Due Date Nov-6 2018

Step 1: Importing necessary libraries for all the necessary modules


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys

### PART1-Algorithmic Implementation
#### 1.Steepest Gradient Descent Method
Details of problem statement for implementing Steepest Gradient Descent Method is available in the file "Computer Project I description.pdf"


In [7]:
class unconstrained_optimizer:
    def __init__(self):
        return
    
    
    #function is the mathematical function that needs to be optimized
    #gradient is the gadient of the mathematical function.
    #line_search is method of finding alpha
    def Steepest_Gradient_Descent_Method(self,function,gradient,line_search,
                                         linear_first_derivative,condition_check,
                                         epsilon,Iter_max,seed_x,linear_second_derivative=None,condition_param=0.5,seed_alpha=0.5):
        #List to store values to be printed at each iteration
        grad_value=[]
        x_value=[]
        alpha_value=[]
        next_x=np.array(seed_x)
        current_alpha_value=seed_alpha
        for i in range(Iter_max):
            print("Iteration:")
            print(i)
            print("Condition Param:")
            print(condition_param)
            current_x=next_x
            current_grad_value=gradient(current_x)
            x_value.append(current_x)
            grad_value.append(current_grad_value)
            criteria=np.linalg.norm(current_grad_value,2)/(1+np.linalg.norm(current_grad_value,1))
            
            if criteria <= epsilon:
                print("Criteria of epsilon met")
                return[x_value,grad_value,alpha_value,criteria]
            if i==25 and condition_param<1e-3:
                condition_param=0.8
                alpha=0.9
            elif i==50 and condition_param<1e-3:
                condition_param=0.8
                alpha=0.9
            elif i==100 and condition_param<1e-2:
                condition_param=0.8
                alpha=0.9
            elif i==200 and condition_param<1e-1:
                condition_param=0.8
                alpha=0.9
            elif i==300:
                alpha=1.5
            elif i==500 :
                condition_param=0.8
                alpha=1.5
            elif i==700:
                alpha=1.5
            
            p=-current_grad_value
            current_alpha_value=line_search(current_x,p,current_alpha_value,function,current_grad_value, 
                                            linear_first_derivative,linear_second_derivative,condition_check,
                                            (condition_param))
            alpha_value.append(current_alpha_value)
            next_x=current_x+(current_alpha_value*p)
        
        print("Maximum iteration reached.Criteria of epsilon not met")
        current_x=next_x
        current_grad_value=gradient(current_x)
        x_value.append(current_x)
        grad_value.append(current_grad_value)
        criteria=np.linalg.norm(current_grad_value,2)/(1+np.linalg.norm(current_grad_value,1))        
        return[x_value,grad_value,alpha_value,criteria]
    
    
    def Newtons_Method(self,gradient_function,hessian_function,epsilon,Iter_max,seed_x):
        x_value=[]
        grad_value=[]
        direction_value=[]
        next_x=np.array(seed_x)
        for i in range(Iter_max):
            current_x_value=next_x
            current_grad_value=gradient_function(current_x_value)
            criteria=np.linalg.norm(current_grad_value,2)/(1+np.linalg.norm(current_grad_value,1))
            x_value.append(current_x)
            grad_value.append(current_grad_value)
            if criteria <= epsilon:
                print("Criteria of epsilon met")
                return[x_value,grad_value,direction_value,criteria]
            current_hessian_value=hessian_function(current_x_value)
            current_direction_value=np.linalg.solve(current_hessian_value,current_gradient)
            direction_value.append(current_direction_value)
            next_x=current_x_value-current_hessian_value.dot(current_direction_value)
        
        print("Maximum iteration reached.Criteria of epsilon not met")
        current_x=next_x
        x_value.append(current_x)
        return[x_value,grad_value,direction_value,criteria]
    
    def BFGS_Quasi_Newton_Method(self,gradient_function,line_search,epsilon,Iter_max,seed_Hessian,seed_x):
        x_value=[]
        grad_value=[]
        direction_value=[]
        next_x=np.matrix(seed_x).T
        next_B=np.matrix(seed_Hessian)
        next_gradient=gradient_function(next_x)
        for i in range(Iter_max):
            current_x_value=next_x
            current_B=next_B
            current_grad_value=next_gradient
            x_value.append(current_x)
            grad_value.append(current_grad_value)
            criteria=np.linalg.norm(current_grad_value,2)/(1+np.linalg.norm(current_grad_value,1))
            if criteria <= epsilon:
                print("Criteria of epsilon met")
                return[x_value,grad_value,direction_value,criteria]
            current_direction_value=np.linalg.solve(current_B,current_gradient)
            direction_value.append(current_direction_value)
            alpha=line_search(current_x)
            next_x=current_x_value+(alpha*current_direction_value)
            next_gradient=gradient_function(next_x)
            s_k=next_x-current_x
            y_k=next_gradient-current_grad_value
            next_B=current_B-((((current_B*s_k)*(current_B*s_k).T)/(s_k.T*current_B*s_k)))+((y_k*y_k.T)/(y_k.T*s_k))
        
        print("Maximum iteration reached.Criteria of epsilon not met")
        current_x=next_x
        x_value.append(current_x)
        return[x_value,grad_value,direction_value,criteria]
    
    def Conjugate_Gradient_Descent_Method(self,line_search,gradient_function,epsilon,Iter_max,seed_x):
        x_value=[]
        grad_value=[]
        direction_value=[]
        next_x=np.matrix(seed_x).T
        next_gradient=np.matrix(gradient_function(next_x)).T
        next_direction=-1*next_gradient
        
        for i in range(Iter_max):
            current_x=next_x
            current_gradient=next_gradient
            current_direction=next_direction
            x_value.append(current_x)
            grad_value.append(current_gradient)
            criteria=np.linalg.norm(current_gradient,2)/(1+np.linalg.norm(current_gradient,1))
            
            if criteria <= epsilon:
                print("Criteria of epsilon met")
                return[x_value,grad_value,direction_value,criteria]
            alpha=line_search(current_x)
            next_x=current_x+alpha*current_direction
            next_gradient=np.matrix(gradient_function(next_x)).T
            beta=np.linalg.norm(current_gradient,2)**2/np.linalg.norm(next_gradient,2)
            next_direction=-1*next_gradient+(beta*current_direction)
        
        print("Maximum iteration reached.Criteria of epsilon not met")
        current_x=next_x
        x_value.append(current_x)
        return[x_value,grad_value,direction_value,criteria]
    

def Newton_line_search(x,p,alpha,function,grad_val,linear_first_derivative,linear_second_derivative,condition_check,condition_param):
    next_alpha=alpha
    condition=condition_check(condition_param,function,grad_val,x,p,next_alpha)
    while(condition==False):
        current_alpha=next_alpha
        first_deri=linear_first_derivative(current_alpha,x,p)
        second_deri=linear_second_derivative(current_alpha,x,p)
        next_alpha=current_alpha-(first_deri/second_deri)
        condition=condition_check(condition_param,function,grad_val,x,p,next_alpha)
        
    return next_alpha

#def secant_line_search(x,p,alpha,function,linear_first_derivative,linear_second_derivative,condition_check,condition_param):
#    condition=False
#    next_alpha=alpha
#    while(condition==False):
#        current_alpha=next_alpha
#        first_deri=linear_first_derivative(current_alpha,x,p)
#        second_deri=linear_second_derivative(current_alpha,x,p)
#        next_alpha=current_alpha-(first_deri/second_deri)
#        condition=condition_check(condition_param,function,x,p,next_alpha)
#        
#    return next_alpha   

#def Wolfe_Condition(condition_param,function,x,p,next_alpha):
#    pass

def Armijo_Condition(condition_param,function,grad_val,x,p,alpha):
    LHS=function(x+alpha*p)
    RHS=function(x)+(condition_param*alpha*np.dot(p,grad_val))
    if LHS<=RHS:
        return True
    else:
        return False
    


In [8]:
def function_1_x(x):
    if len(x)!=3:
        sys.exit("fuction1 can handle only vector of size 3")
    else:
        return x[0]**2+x[1]**2+x[2]**2

def grad_function_1_x(x):
    if len(x)!=3:
        sys.exit("fuction1 can handle only vector of size 3")
    else:
        return np.array([2*x[0],2*x[1],2*x[2]])

def linear_first_derivative_function_1_alpha(alpha,x,p):
    return 2*np.dot(x+alpha*p,p) 

def linear_second_derivative_function_1_alpha(alpha,x,p):
    return 2*np.dot(p,p)

optimize=unconstrained_optimizer()
[xval,graval,alval,crit]=optimize.Steepest_Gradient_Descent_Method(function_1_x,grad_function_1_x,
                                                                          Newton_line_search,linear_first_derivative_function_1_alpha,
                                                                          Armijo_Condition,
                                                                          1e-5,1000,np.array([1,1,1]),linear_second_derivative_function_1_alpha,
                                                                          0.5,0.5)
#print('Values of x :')
#print(xval)
#print('Values of func :')
#print(funval)
#print('Values of grad :')
#print(graval)
#print('Values of alpha :')
#print(alval)
print('Values of crit :')
print(crit)
print('last value of x:')
print(xval[-1])

Iteration:
0
Condition Param:
0.5
Iteration:
1
Condition Param:
0.5
Criteria of epsilon met
Values of crit :
0.0
last value of x:
[0. 0. 0.]


In [9]:
def function_2_x(x):
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return x[0]**2+2*x[1]**2-2*x[0]*x[1]-2*x[1]

def grad_function_2_x(x):
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return np.array([2*x[0]-2*x[1],4*x[1]-2*x[0]-2])

def linear_first_derivative_function_2_alpha(alpha,x,p):
    nx=x+alpha*p
    return 2*nx[0]*p[0]+4*nx[1]*p[1]-2*p[0]*nx[1]-2*p[1]*nx[0]-2*p[1]

def linear_second_derivative_function_2_alpha(alpha,x,p):
    return 2*p[0]**2+4*p[1]**2-4*p[1]*p[0]

optimize=unconstrained_optimizer()
[xval,graval,alval,crit]=optimize.Steepest_Gradient_Descent_Method(function_2_x,grad_function_2_x,
                                                                          Newton_line_search,linear_first_derivative_function_2_alpha,
                                                                          Armijo_Condition,
                                                                          1e-5,1000,np.array([0,0]),linear_second_derivative_function_2_alpha,
                                                                          0.01,1)

print('Values of crit :')
print(crit)
print('last value of x:')
print(xval[-1])

Iteration:
0
Condition Param:
0.01
Iteration:
1
Condition Param:
0.01
Iteration:
2
Condition Param:
0.01
Iteration:
3
Condition Param:
0.01
Iteration:
4
Condition Param:
0.01
Iteration:
5
Condition Param:
0.01
Iteration:
6
Condition Param:
0.01
Iteration:
7
Condition Param:
0.01
Iteration:
8
Condition Param:
0.01
Iteration:
9
Condition Param:
0.01
Iteration:
10
Condition Param:
0.01
Iteration:
11
Condition Param:
0.01
Iteration:
12
Condition Param:
0.01
Iteration:
13
Condition Param:
0.01
Iteration:
14
Condition Param:
0.01
Iteration:
15
Condition Param:
0.01
Iteration:
16
Condition Param:
0.01
Iteration:
17
Condition Param:
0.01
Iteration:
18
Condition Param:
0.01
Iteration:
19
Condition Param:
0.01
Iteration:
20
Condition Param:
0.01
Iteration:
21
Condition Param:
0.01
Iteration:
22
Condition Param:
0.01
Iteration:
23
Condition Param:
0.01
Iteration:
24
Condition Param:
0.01
Iteration:
25
Condition Param:
0.01
Iteration:
26
Condition Param:
0.01
Iteration:
27
Condition Param:
0.01
It

In [12]:
def function_3_x(x):
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return (100*(x[1]-x[0]**2)**2)+((1-x[0])**2)

def grad_function_3_x(x):
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return np.array([200*(x[1]-x[0]**2)*(-2*x[0])+(-2*(1-x[0])),200*(x[1]-x[0]**2)])

def linear_first_derivative_function_3_alpha(alpha,x,p):
    nx=x+alpha*p
    return 200*(nx[1]-nx[0]**2)*(p[1]-2*nx[0]*p[0])-2*(1-nx[0])*p[0]

def linear_second_derivative_function_3_alpha(alpha,x,p):
    nx=x+alpha*p
    return 200*(p[1]-2*nx[0]*p[0])*(-2*p[0]**2)+2*p[0]**2

optimize=unconstrained_optimizer()
[xval,graval,alval,crit]=optimize.Steepest_Gradient_Descent_Method(function_3_x,grad_function_3_x,
                                                                          Newton_line_search,linear_first_derivative_function_3_alpha,
                                                                          Armijo_Condition,
                                                                          1e-5,1000,np.array([-1.2,1]),linear_second_derivative_function_3_alpha,
                                                                          1e-4,1e-3)

print('Values of crit :')
print(crit)
print('last value of x:')
print(xval[-1])

Iteration:
0
Condition Param:
0.0001
Iteration:
1
Condition Param:
0.0001
Iteration:
2
Condition Param:
0.0001
Iteration:
3
Condition Param:
0.0001
Iteration:
4
Condition Param:
0.0001
Iteration:
5
Condition Param:
0.0001
Iteration:
6
Condition Param:
0.0001
Iteration:
7
Condition Param:
0.0001
Iteration:
8
Condition Param:
0.0001
Iteration:
9
Condition Param:
0.0001
Iteration:
10
Condition Param:
0.0001
Iteration:
11
Condition Param:
0.0001
Iteration:
12
Condition Param:
0.0001
Iteration:
13
Condition Param:
0.0001
Iteration:
14
Condition Param:
0.0001
Iteration:
15
Condition Param:
0.0001
Iteration:
16
Condition Param:
0.0001
Iteration:
17
Condition Param:
0.0001
Iteration:
18
Condition Param:
0.0001
Iteration:
19
Condition Param:
0.0001
Iteration:
20
Condition Param:
0.0001
Iteration:
21
Condition Param:
0.0001
Iteration:
22
Condition Param:
0.0001
Iteration:
23
Condition Param:
0.0001
Iteration:
24
Condition Param:
0.0001
Iteration:
25
Condition Param:
0.0001
Iteration:
26
Conditio

Iteration:
350
Condition Param:
0.8
Iteration:
351
Condition Param:
0.8
Iteration:
352
Condition Param:
0.8
Iteration:
353
Condition Param:
0.8
Iteration:
354
Condition Param:
0.8
Iteration:
355
Condition Param:
0.8
Iteration:
356
Condition Param:
0.8
Iteration:
357
Condition Param:
0.8
Iteration:
358
Condition Param:
0.8
Iteration:
359
Condition Param:
0.8
Iteration:
360
Condition Param:
0.8
Iteration:
361
Condition Param:
0.8
Iteration:
362
Condition Param:
0.8
Iteration:
363
Condition Param:
0.8
Iteration:
364
Condition Param:
0.8
Iteration:
365
Condition Param:
0.8
Iteration:
366
Condition Param:
0.8
Iteration:
367
Condition Param:
0.8
Iteration:
368
Condition Param:
0.8
Iteration:
369
Condition Param:
0.8
Iteration:
370
Condition Param:
0.8
Iteration:
371
Condition Param:
0.8
Iteration:
372
Condition Param:
0.8
Iteration:
373
Condition Param:
0.8
Iteration:
374
Condition Param:
0.8
Iteration:
375
Condition Param:
0.8
Iteration:
376
Condition Param:
0.8
Iteration:
377
Condition Par

Iteration:
638
Condition Param:
0.8
Iteration:
639
Condition Param:
0.8
Iteration:
640
Condition Param:
0.8
Iteration:
641
Condition Param:
0.8
Iteration:
642
Condition Param:
0.8
Iteration:
643
Condition Param:
0.8
Iteration:
644
Condition Param:
0.8
Iteration:
645
Condition Param:
0.8
Iteration:
646
Condition Param:
0.8
Iteration:
647
Condition Param:
0.8
Iteration:
648
Condition Param:
0.8
Iteration:
649
Condition Param:
0.8
Iteration:
650
Condition Param:
0.8
Iteration:
651
Condition Param:
0.8
Iteration:
652
Condition Param:
0.8
Iteration:
653
Condition Param:
0.8
Iteration:
654
Condition Param:
0.8
Iteration:
655
Condition Param:
0.8
Iteration:
656
Condition Param:
0.8
Iteration:
657
Condition Param:
0.8
Iteration:
658
Condition Param:
0.8
Iteration:
659
Condition Param:
0.8
Iteration:
660
Condition Param:
0.8
Iteration:
661
Condition Param:
0.8
Iteration:
662
Condition Param:
0.8
Iteration:
663
Condition Param:
0.8
Iteration:
664
Condition Param:
0.8
Iteration:
665
Condition Par

0.8
Iteration:
975
Condition Param:
0.8
Iteration:
976
Condition Param:
0.8
Iteration:
977
Condition Param:
0.8
Iteration:
978
Condition Param:
0.8
Iteration:
979
Condition Param:
0.8
Iteration:
980
Condition Param:
0.8
Iteration:
981
Condition Param:
0.8
Iteration:
982
Condition Param:
0.8
Iteration:
983
Condition Param:
0.8
Iteration:
984
Condition Param:
0.8
Iteration:
985
Condition Param:
0.8
Iteration:
986
Condition Param:
0.8
Iteration:
987
Condition Param:
0.8
Iteration:
988
Condition Param:
0.8
Iteration:
989
Condition Param:
0.8
Iteration:
990
Condition Param:
0.8
Iteration:
991
Condition Param:
0.8
Iteration:
992
Condition Param:
0.8
Iteration:
993
Condition Param:
0.8
Iteration:
994
Condition Param:
0.8
Iteration:
995
Condition Param:
0.8
Iteration:
996
Condition Param:
0.8
Iteration:
997
Condition Param:
0.8
Iteration:
998
Condition Param:
0.8
Iteration:
999
Condition Param:
0.8
Maximum iteration reached.Criteria of epsilon not met
Values of crit :
0.43995249041784396
last 

In [ ]:
def function_4_x(x):
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return (x[0]+x[1])**4 + x[1]**2

def grad_function_4_x(x):
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return np.array([4*(x[0]+x[1])**3,4*(x[0]+x[1])**3+ 2*x[1]])

def linear_first_derivative_function_4_alpha(alpha,x,p):
    nx=x+alpha*p
    return (4*((nx[0]+nx[1])**3)*(p[0]+p[1]))+2*nx[1]*p[1]

def linear_second_derivative_function_4_alpha(alpha,x,p):
    nx=x+alpha*p
    return (12*((nx[0]+nx[1])**2)*(p[0]+p[1])**2)+2*p[1]**2

optimize=unconstrained_optimizer()
[xval,graval,alval,crit]=optimize.Steepest_Gradient_Descent_Method(function_4_x,grad_function_4_x,
                                                                          Newton_line_search,linear_first_derivative_function_4_alpha,
                                                                          Armijo_Condition,
                                                                          1e-5,1000,np.array([2,-2]),linear_second_derivative_function_4_alpha,
                                                                          0.5,0.5)

print('Values of crit :')
print(crit)
print('last value of x:')
print(xval[-1])

In [ ]:
def function_5_x(x,c=1):
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return (x[0]-1)**2+(x[1]-1)**2+c*(x[0]**2+x[1]**2-0.25)**2

def grad_function_5_x(x,c=1):
    if len(x)!=2:
        sys.exit("fuction1 can handle only vector of size 2")
    else:
        return np.array([(2*(x[0]-1))+(2*c*(x[0]**2+x[1]**2-0.25)*2*x[0]),(2*(x[1]-1))+(2*c*(x[0]**2+x[1]**2-0.25)*2*x[1])])

def linear_first_derivative_function_5_alpha(alpha,x,p,c=1):
    nx=x+alpha*p
    return 2*nx[0]*p[0]+2*nx[1]*p[1]+(2*c*(nx[0]**2+nx[1]**2-0.25)*(2*nx[0]*p[0]+2*nx[1]*p[1]))

def linear_second_derivative_function_5_alpha(alpha,x,p,c=1):
    nx=x+alpha*p
    return 2*p[0]**2+2*p[1]**2+(2*c*(2*nx[0]*p[0]+2*nx[1]*p[1])**2)+(2*c*(nx[0]**2+nx[1]**2-0.25)*2*p[0]**2+2*p[1]**2)

optimize=unconstrained_optimizer()
[xval,graval,alval,crit]=optimize.Steepest_Gradient_Descent_Method(function_5_x,grad_function_5_x,
                                                                          Newton_line_search,linear_first_derivative_function_5_alpha,
                                                                          Armijo_Condition,
                                                                          1e-5,1000,np.array([1,-1]),linear_second_derivative_function_5_alpha,
                                                                          0.5,0.5)

print('Values of crit :')
print(crit)
print('last value of x:')
print(xval[-1])